In [1]:
import numpy as np
import pandas as pd
import sklearn.gaussian_process as gp
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder
from numpy.random import RandomState

from util.defs import ParamType
from util.util import random_x_sample
from acquisition.acquisition_functions import ProbabilityOfImprovement, ExpectedImprovement
from acquisition.acquisition_optimizers import RandomAcquisitionOpt, ConstrainedAcquisitionOpt
from hp_optimizers.hp_optimizer import GPROptimizer, RandomSearchOptimizer

# Set random seed

In [2]:
rand = RandomState(seed=123)

# Grab the dataset

In [3]:
income = "../data/income.csv"
df = pd.read_csv(income)
data = df.values
rand.shuffle(data)
X = data[:, :-1]
y = data[:, -1]

In [4]:
print(data.shape)
print(df.shape)


(30162, 105)
(30162, 105)


# Define: bounds for hyperparameters, parameter data types, objective function (lower is better)

In [5]:
# x0 = num layers [>1]
# x1 = learning rate init [0-1]
# x2 = alpha [0-1]
# x3 = beta1 [0-1]
# x4 = beta2 [0-1]
bounds = np.array([
    [2, 100],
    [0.001, 1 - 0.001],
    [0, 0.5],
    [0.8, 1 - 0.000001],
    [0.8, 1 - 0.000001]
])

p_types = [ParamType.Disc, ParamType.Cont, ParamType.Cont, ParamType.Cont, ParamType.Cont]

def objective(x):
    # x may be passed as a (1, p) array
    if x.ndim > 1:
        x = x[0]
    # define model
    model = MLPClassifier(hidden_layer_sizes=int(x[0]),
                          learning_rate_init=x[1],
                          alpha=x[2],
                          random_state=rand,
                          max_iter = 100
                         )

    # get 5-fold cross validation results
    cv_res = cross_validate(model, X, y, cv=5)
    # return 1 - acc
    return 1 - cv_res["test_score"].mean()

# Define the Bayesian Optimizer

In [6]:
kernel = gp.kernels.RBF()
gpr = gp.GaussianProcessRegressor(kernel,
                                  alpha=0.5,
                                  #alpha=np.abs(rand.normal(scale=.1)),
                                  #alpha=np.abs(rand.normal(scale=.01)),
                                  n_restarts_optimizer=20, 
                                  random_state=rand)
acquisition = ExpectedImprovement()
# acquisition = ProbabilityOfImprovement()
opt_acquisition = ConstrainedAcquisitionOpt(gpr, acquisition, rand)
gpr_opt = GPROptimizer(gpr, opt_acquisition, objective, bounds, p_types, rand, initial_samples=3, fit=True)

/home/brendan/anaconda/envs/cs446-gp-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Run the optimization

In [7]:
res1 = gpr_opt.optimize(10, thresh=0.03, verbose=True)
print(gpr_opt.X)
print(gpr_opt.y)

Optimization iteration 1
Selected next parameter sample from acquisition optimizer: [26.          0.14016233  0.29287665  0.82115043  0.9340854 ]


/home/brendan/anaconda/envs/cs446-gp-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/brendan/anaconda/envs/cs446-gp-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Objective value at sample: 0.2471

Optimization iteration 2
Selected next parameter sample from acquisition optimizer: [23.          0.62945793  0.10939049  0.89335251  0.86275202]


/home/brendan/anaconda/envs/cs446-gp-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Objective value at sample: 0.2487

Optimization iteration 3
Selected next parameter sample from acquisition optimizer: [24.          0.4547958   0.2064646   0.91088278  0.9835588 ]
Objective value at sample: 0.2489

Optimization iteration 4
Selected next parameter sample from acquisition optimizer: [29.          0.61217223  0.12032455  0.8317524   0.99721892]


...

Objective value at sample: 0.2489

Optimization iteration 7
Selected next parameter sample from acquisition optimizer: [20.          0.81742407  0.05372186  0.95640106  0.9583118 ]


/home/brendan/anaconda/envs/cs446-gp-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Objective value at sample: 0.2489

Optimization iteration 8
Selected next parameter sample from acquisition optimizer: [32.          0.55678276  0.03797748  0.99483069  0.97671198]
Objective value at sample: 0.2489

Optimization iteration 9
Selected next parameter sample from acquisition optimizer: [30.          0.18728035  0.32635182  0.90963805  0.89780154]


/home/brendan/anaconda/envs/cs446-gp-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/brendan/anaconda/envs/cs446-gp-env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Objective value at sample: 0.2488

Optimization iteration 10
Selected next parameter sample from acquisition optimizer: [2.90000000e+01 4.41245079e-02 4.15376094e-03 9.74720948e-01
 9.23619423e-01]
Objective value at sample: 0.2174

Optimization yielded: {'argmin': 0, 'minimizer': array([2.00000000e+01, 1.05154652e-02, 1.97165282e-01, 8.07757167e-01,
       8.23545815e-01]), 'minimum': array([0.20237366])}
[[2.00000000e+01 1.05154652e-02 1.97165282e-01 8.07757167e-01
  8.23545815e-01]
 [4.90000000e+01 9.33537776e-02 4.46793769e-01 9.45403253e-01
  8.68260928e-01]
 [2.30000000e+01 6.03212030e-01 8.21767003e-02 9.99371451e-01
  8.62532700e-01]
 [2.60000000e+01 1.40162335e-01 2.92876645e-01 8.21150425e-01
  9.34085405e-01]
 [2.30000000e+01 6.29457934e-01 1.09390491e-01 8.93352510e-01
  8.62752019e-01]
 [2.40000000e+01 4.54795804e-01 2.06464599e-01 9.10882780e-01
  9.83558797e-01]
 [2.90000000e+01 6.12172233e-01 1.20324547e-01 8.31752397e-01
  9.97218924e-01]
 [1.60000000e+01 8.59237511e-0

In [ ]:
res1

# Try Random Search for comparison

In [ ]:
rand_opt = RandomSearchOptimizer(objective, bounds, p_types, rand, parallel=False)
res2 = rand_opt.optimize(13, verbose=False)
res2